In [36]:
import pandas as pd
import folium
from folium.plugins import HeatMap

### Creating Map

In [37]:
# creating map at portland coordinates
portland_coordinates = [45.523, -122.676]
map = folium.Map(location= portland_coordinates, 
                 zoom_start=10, 
                 control_scale=True)

# displaying map
map

### Loading Air Quality Data

In [38]:
air_df = pd.read_csv('./Data/air_polution.csv')
air_df = air_df.drop(columns = ['Date'])
air_df = air_df.rename(columns={'Date_Converted': 'Date'}).sort_values(by = ['Date'], ascending=False).reset_index(drop=True)
air_df.head(10)

,Value,location,parameter,longitude,latitude,Date
0,0.837500,1552754,pm25,-122.432269,45.592266,2024-08-22 13:00:00+00:00
1,0.266369,2923949,pm25,-122.627171,45.552272,2024-08-22 13:00:00+00:00
2,-1.000000,1526,pm25,-122.587370,45.643360,2024-08-22 12:00:00+00:00
3,2.800000,1064,pm25,-122.602877,45.496641,2024-08-22 12:00:00+00:00
4,2.200000,8744,pm25,-122.575624,45.562203,2024-08-22 12:00:00+00:00
5,0.452381,2923949,pm25,-122.627171,45.552272,2024-08-22 12:00:00+00:00
6,2.000000,8573,pm25,-122.483400,45.496200,2024-08-22 12:00:00+00:00
7,0.691667,1552754,pm25,-122.432269,45.592266,2024-08-22 12:00:00+00:00
8,2.000000,1502,pm25,-122.815900,45.470200,2024-08-22 12:00:00+00:00
9,2.800000,1067,pm25,-122.772100,45.768531,2024-08-22 12:00:00+00:00


### Plotting Points
Performing basic plotting of each of the points

In [39]:
def plotPoint(row, map):

    '''Takes row in dataframe and folium map object and plots latitude/longitude
    point in dataframe onto map object'''

    folium.CircleMarker(location=[row.latitude, row.longitude],
                        radius=4,
                        weight=row.Value).add_to(map)

In [40]:
# Creating datafrom of last measurement from each location
number_of_locations = len(air_df['location'].drop_duplicates())
last_observation_df = air_df.head(number_of_locations)

# Iterating through dataframe and plotting points
for index, row in last_observation_df.iterrows():
    plotPoint(row, map)

map

### Performing Interpolation Between Points

In [41]:
# Creating second map to use for heatmap
second_map = folium.Map(location= portland_coordinates, 
                 zoom_start=10,
                 control_scale=True)
# Converting points to numpy array
test = last_observation_df[['latitude', 'longitude', 'Value']].to_numpy()

In [42]:
# Adding points to map and displaying
HeatMap(test).add_to(second_map)
second_map

### Adding a Time Dimension

In [53]:
# Assembling timestamped data
dates = air_df['Date'].drop_duplicates().sort_values(ascending=False).to_list()
locations = air_df['location'].drop_duplicates().sort_values(ascending=False).to_list()

time_ordered = []
for i in dates:
    timestamp = air_df[air_df['Date'] == i][['latitude', 'longitude', 'Value']].to_numpy().tolist()
    time_ordered.append(timestamp)
    time_ordered.append(timestamp)
print(time_ordered)

[[[45.592266, -122.432269, 0.837499999], [45.552272, -122.627171, 0.266369053]], [[45.592266, -122.432269, 0.837499999], [45.552272, -122.627171, 0.266369053]], [[45.64336, -122.58737, -1.0], [45.496641, -122.602877, 2.8], [45.562203, -122.575624, 2.2], [45.552272, -122.627171, 0.45238096], [45.4962, -122.4834, 2.0], [45.592266, -122.432269, 0.691666665], [45.4702, -122.8159, 2.0], [45.768531, -122.7721, 2.8], [45.25928, -122.588151, 2.7]], [[45.64336, -122.58737, -1.0], [45.496641, -122.602877, 2.8], [45.562203, -122.575624, 2.2], [45.552272, -122.627171, 0.45238096], [45.4962, -122.4834, 2.0], [45.592266, -122.432269, 0.691666665], [45.4702, -122.8159, 2.0], [45.768531, -122.7721, 2.8], [45.25928, -122.588151, 2.7]], [[45.64336, -122.58737, -1.0], [45.4702, -122.8159, 2.0], [45.552272, -122.627171, 0.239583337], [45.768531, -122.7721, 2.8], [45.562203, -122.575624, 2.1], [45.51421, -122.653294, 1.8], [45.496641, -122.602877, 2.7], [45.25928, -122.588151, 2.6], [45.4962, -122.4834, 1.

In [55]:
m = folium.Map(location= portland_coordinates, 
                 zoom_start=10, 
                 control_scale=True)
hm = folium.plugins.HeatMapWithTime(time_ordered)
hm.add_to(m)
m